In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from langchain.document_loaders import CSVLoader

from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.llms import Ollama

In [5]:
loader = CSVLoader('data_input/test2.csv')
big_df = loader.load()

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 5000,
    chunk_overlap = 100,
    length_function = len,
    add_start_index = True
)

In [7]:
chunks = text_splitter.split_documents(big_df)

In [12]:
len(chunks)

26

In [10]:
embedings = OllamaEmbeddings(
    model='all-minilm'
)

In [11]:
chromadb = Chroma.from_documents(chunks, embedings, persist_directory = 'vector_db', collection_name= 'vectors_test')

In [15]:
# Prompt
template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Conversation_history: {conversation_history}

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)

In [16]:
# Chain
MODEL = 'llama2'
model = Ollama(model=MODEL)
parser = StrOutputParser()
retriever = chromadb.as_retriever()

from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "conversation_history": itemgetter("conversation_history"),
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [17]:
query = 'why month was first called Sextilis in Latin?'
history = ''

answer = chain.invoke({'question': query, "conversation_history": history})
print(answer)

The reason why April was first called Sextilis in Latin is not definitively known, as the origins of Roman calendar names are not well-documented. However, there are several theories:

1. Connection to the six days of the Roman week: In ancient Rome, the month was named after the sixth day of the Roman week, which was called "Sexta." Therefore, it is possible that the name Sextilis was chosen because it was associated with the number six.
2. Association with the star Sirius: The Romans believed that the star Sirius, also known as the Dog Star, rose on the sixth day after the spring equinox. As a result, some scholars suggest that Sextilis may have been named after this star.
3. Connection to the goddess Venus: Some historians believe that Sextilis was named after the Roman goddess of love and beauty, Venus, who was associated with the sixth day of the week.
4. Lunar cycle: Another theory is that Sextilis was named after the lunar cycle, with the sixth month corresponding to the start o